In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
cluster = pd.read_csv('data/중극장_군집분석.csv', encoding='cp949')
recover = pd.read_csv('data/충격지점이후분석.csv')

## Cluster 값 조인

In [ ]:
cluster= recover.merge(cluster[['공연시설코드', 'cluster']], on='공연시설코드', how='left')

In [ ]:
cluster

,공연시설코드,충격반응력,충격회복력,cluster
0,FC222007,0.999441,838.051600,1.0
1,FC222040,0.887045,2.596457,0.0
2,FC222048,0.996858,53.153837,1.0
3,FC222049,0.000000,0.000000,0.0
4,FC222054,0.965359,2.864677,0.0
...,...,...,...,...
133,FC223909,0.991397,16.383527,0.0
134,FC223941,0.000000,0.000000,1.0
135,FC224381,0.000000,0.000000,1.0
136,FC224454,0.000000,0.000000,1.0


In [ ]:
data = cluster[(cluster['충격회복력']!=0)&(cluster['충격회복력']!=-100)]
minvalue = data.groupby('cluster')[['충격회복력', '충격반응력']].min()
minvalue

,충격회복력,충격반응력
cluster,,
0.0,0.090933,0.114183
1.0,0.542680,0.250000


In [ ]:
c0_recovery_min=minvalue.iloc[0,0]
c0_response_min=minvalue.iloc[0,1]
c1_recovery_min=minvalue.iloc[1,0]
c1_response_min=minvalue.iloc[1,1]

In [ ]:
final_ratio=[]
theaters = cluster['공연시설코드'].unique()
for theater in theaters:
    if cluster[cluster['공연시설코드']==theater]['cluster'].iloc[0]==0:
        ratio=math.log((cluster[cluster['공연시설코드']==theater]['충격회복력']-c0_recovery_min+0.0001)/(cluster[cluster['공연시설코드']==theater]['충격반응력']-c0_response_min+0.0001))
    else:
        ratio=math.log((cluster[cluster['공연시설코드']==theater]['충격회복력']-c1_recovery_min+0.0001)/(cluster[cluster['공연시설코드']==theater]['충격반응력']-c1_response_min+0.0001))
    final_ratio.append(ratio)
cluster['ratio']=final_ratio


C:\Users\ohke7\AppData\Local\Temp\ipykernel_18232\3992247558.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ratio=math.log((cluster[cluster['공연시설코드']==theater]['충격회복력']-c1_recovery_min+0.0001)/(cluster[cluster['공연시설코드']==theater]['충격반응력']-c1_response_min+0.0001))
C:\Users\ohke7\AppData\Local\Temp\ipykernel_18232\3992247558.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ratio=math.log((cluster[cluster['공연시설코드']==theater]['충격회복력']-c0_recovery_min+0.0001)/(cluster[cluster['공연시설코드']==theater]['충격반응력']-c0_response_min+0.0001))


In [ ]:
data = cluster[(cluster['충격회복력']!=0)&(cluster['충격회복력']!=-100)]
ratiosummary = data.groupby('cluster')[['ratio']].agg(['mean', 'std'])
ratiosummary

ratio          
             mean       std
cluster                    
0.0      2.289903  1.696174
1.0      2.384379  2.853239

In [ ]:
c0_aver=ratiosummary.iloc[0,0]
c0_stdev=ratiosummary.iloc[0,1]
c1_aver=ratiosummary.iloc[1,0]
c1_stdev=ratiosummary.iloc[1,1]

In [ ]:
final_resilience=[]
theaters = cluster['공연시설코드'].unique()
for theater in theaters:
    if cluster[cluster['공연시설코드']==theater]['cluster'].iloc[0]==0:
        resilience=(cluster[cluster['공연시설코드']==theater]['ratio']-c0_aver)/c0_stdev
    else:
        resilience=(cluster[cluster['공연시설코드']==theater]['ratio']-c1_aver)/c1_stdev
    final_resilience.append(resilience.iloc[0])
cluster['resilience']=final_resilience


In [ ]:
cluster

,공연시설코드,감소폭,감소기간,최저지점이후회복기간,총기간,충격반응력,충격회복력,cluster,ratio,resilience
0,FC222007,17867850.0,15,6,21,0.999441,838.051600,1.0,7.018727,1.624241
1,FC222040,3548180.0,12,9,21,0.887045,2.596457,0.0,1.176063,-0.656679
2,FC222048,31731533.0,2,12,14,0.996858,53.153837,1.0,4.254676,0.655500
3,FC222049,0.0,0,0,0,0.000000,0.000000,0.0,-0.227898,-1.484401
4,FC222054,10366874.0,4,10,14,0.965359,2.864677,0.0,1.181252,-0.653619
...,...,...,...,...,...,...,...,...,...,...
133,FC223909,108654235.0,4,22,26,0.991397,16.383527,0.0,2.921606,0.372428
134,FC223941,0.0,0,0,0,0.000000,0.000000,1.0,0.775274,-0.563957
135,FC224381,0.0,0,0,0,0.000000,0.000000,1.0,0.775274,-0.563957
136,FC224454,0.0,0,0,0,0.000000,0.000000,1.0,0.775274,-0.563957


In [ ]:
# cluster.to_csv("Medium_Theater_0_Reflection_X.csv", index=False)

In [ ]:
data = cluster[(cluster['충격회복력']!=0)&(cluster['충격회복력']!=-100)&(np.isnan(cluster['cluster'])==False)]
ratiosummary = data.groupby('cluster')[['감소폭', '감소기간', '최저지점이후회복기간', '총기간', '충격반응력', '충격회복력', 'ratio','resilience']].agg('mean')
ratiosummary

,감소폭,감소기간,최저지점이후회복기간,총기간,충격반응력,충격회복력,ratio,resilience
cluster,,,,,,,,
0.0,1.852476e+07,7.244444,8.733333,15.977778,0.862335,35.339623,2.289903,-1.690006e-16
1.0,9.605339e+06,11.256410,6.743590,18.000000,0.866736,37.966301,2.384379,-1.441155e-16


In [ ]:
ratiosummary.iloc[0,7] #표준화를 했기 때문에 0에 가까운 수치임

-1.6900061597071826e-16

# 회복점수구하기

## original

In [ ]:
import numpy as np
all_scores=[]

scores=data['resilience']
max_score = max(scores)
min_score = min(scores)
scaled_scores = [(score - min_score) / (max_score - min_score) * 100 for score in scores]
percentiles = [np.percentile(scaled_scores, (score - min_score) / (max_score - min_score) * 100) for score in scores]
all_scores.append(percentiles)

for score, percentile in zip(scores, percentiles):
    print(f"Score: {score}, Percentile: {percentile}")


Score: 1.6242411612883911, Percentile: 65.12589465628491
Score: -0.6566785657693849, Percentile: 55.790878529187424
Score: 0.655499518882575, Percentile: 61.01679282930905
Score: -0.6536190531985638, Percentile: 55.79490663846433
Score: -1.2507388914953785, Percentile: 52.98177699759682
Score: 0.47586184914747603, Percentile: 60.23399828860748
Score: -1.1614577279510603, Percentile: 53.196548542820594
Score: 0.6908200217883215, Percentile: 61.48427463054757
Score: 0.03399366762309262, Percentile: 58.64072410218873
Score: 0.03771705925043745, Percentile: 58.64612581168582
Score: 0.6173549664889608, Percentile: 60.5119335381492
Score: -0.419915747734688, Percentile: 56.24226151800847
Score: 3.0460793607526973, Percentile: 100.0
Score: 0.7868752696785185, Percentile: 61.9851988983364
Score: -0.18701484914099223, Percentile: 56.79107284290377
Score: -0.2187151097005295, Percentile: 56.77613958498761
Score: 1.682035951978928, Percentile: 65.50367281052577
Score: -0.8489755784826477, Percent

In [ ]:
data['회복점수']=all_scores[0]

C:\Users\ohke7\AppData\Local\Temp\ipykernel_18232\871335482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['회복점수']=all_scores[0]


In [ ]:
data

,공연시설코드,감소폭,감소기간,최저지점이후회복기간,총기간,충격반응력,충격회복력,cluster,ratio,resilience,회복점수
0,FC222007,17867850.0,15,6,21,0.999441,838.051600,1.0,7.018727,1.624241,65.125895
1,FC222040,3548180.0,12,9,21,0.887045,2.596457,0.0,1.176063,-0.656679,55.790879
2,FC222048,31731533.0,2,12,14,0.996858,53.153837,1.0,4.254676,0.655500,61.016793
4,FC222054,10366874.0,4,10,14,0.965359,2.864677,0.0,1.181252,-0.653619,55.794907
6,FC222061,851200.0,3,14,17,0.928042,1.054113,0.0,0.168433,-1.250739,52.981777
...,...,...,...,...,...,...,...,...,...,...,...
129,FC223835,12459193.0,16,1,17,0.771788,3.490741,0.0,1.642747,-0.381539,56.484193
130,FC223889,43647087.0,21,1,22,0.981118,112.850000,0.0,4.867930,1.519907,64.503584
131,FC223890,979125.0,3,25,28,0.757177,1.012102,0.0,0.359460,-1.138117,53.252181
132,FC223897,5539113.0,9,5,14,0.790950,5.823907,1.0,2.278421,-0.037136,58.087509


In [ ]:
data.groupby('cluster')['회복점수'].agg(['sum'])


,sum
cluster,
0.0,2616.841216
1.0,2258.876440


## 회복이 안된 극장들이 2022년 12월에 회복이 모두 됐을때

In [ ]:
cluster = pd.read_csv('data/중극장_군집분석.csv', encoding='cp949')
recover = pd.read_csv("data/회복모두완료된극장의분석.csv")

In [ ]:
cluster= recover.merge(cluster[['공연시설코드', 'cluster']], on='공연시설코드', how='left')

In [ ]:
cluster

,공연시설코드,충격반응력,충격회복력,cluster
0,FC222007,0.999441,838.051600,1.0
1,FC222040,0.887045,2.596457,0.0
2,FC222048,0.996858,53.153837,1.0
3,FC222049,0.000000,0.000000,0.0
4,FC222054,0.965359,2.864677,0.0
...,...,...,...,...
133,FC223909,0.991397,16.383527,0.0
134,FC223941,0.000000,0.000000,1.0
135,FC224381,0.000000,0.000000,1.0
136,FC224454,0.000000,0.000000,1.0


In [ ]:
data = cluster[(cluster['충격회복력']!=0)&(cluster['충격회복력']!=-100)]
minvalue = data.groupby('cluster')[['충격회복력', '충격반응력']].min()
minvalue

,충격회복력,충격반응력
cluster,,
0.0,0.090933,0.114183
1.0,0.165179,0.250000


In [ ]:
c0_recovery_min=minvalue.iloc[0,0]
c0_response_min=minvalue.iloc[0,1]
c1_recovery_min=minvalue.iloc[1,0]
c1_response_min=minvalue.iloc[1,1]

In [ ]:
final_ratio=[]
theaters = cluster['공연시설코드'].unique()
for theater in theaters:
    if cluster[cluster['공연시설코드']==theater]['cluster'].iloc[0]==0:
        ratio=math.log((cluster[cluster['공연시설코드']==theater]['충격회복력']-c0_recovery_min+0.0001)/(cluster[cluster['공연시설코드']==theater]['충격반응력']-c0_response_min+0.0001))
    else:
        ratio=math.log((cluster[cluster['공연시설코드']==theater]['충격회복력']-c1_recovery_min+0.0001)/(cluster[cluster['공연시설코드']==theater]['충격반응력']-c1_response_min+0.0001))
    final_ratio.append(ratio)
cluster['ratio']=final_ratio


C:\Users\ohke7\AppData\Local\Temp\ipykernel_18232\3992247558.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ratio=math.log((cluster[cluster['공연시설코드']==theater]['충격회복력']-c1_recovery_min+0.0001)/(cluster[cluster['공연시설코드']==theater]['충격반응력']-c1_response_min+0.0001))
C:\Users\ohke7\AppData\Local\Temp\ipykernel_18232\3992247558.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ratio=math.log((cluster[cluster['공연시설코드']==theater]['충격회복력']-c0_recovery_min+0.0001)/(cluster[cluster['공연시설코드']==theater]['충격반응력']-c0_response_min+0.0001))


In [ ]:
data = cluster[(cluster['충격회복력']!=0)&(cluster['충격회복력']!=-100)]
ratiosummary = data.groupby('cluster')[['ratio']].agg(['mean', 'std'])
ratiosummary

ratio          
             mean       std
cluster                    
0.0      2.017348  1.871015
1.0      2.254063  2.723920

In [ ]:
c0_aver=ratiosummary.iloc[0,0]
c0_stdev=ratiosummary.iloc[0,1]
c1_aver=ratiosummary.iloc[1,0]
c1_stdev=ratiosummary.iloc[1,1]

In [ ]:
final_resilience=[]
theaters = cluster['공연시설코드'].unique()
for theater in theaters:
    if cluster[cluster['공연시설코드']==theater]['cluster'].iloc[0]==0:
        resilience=(cluster[cluster['공연시설코드']==theater]['ratio']-c0_aver)/c0_stdev
    else:
        resilience=(cluster[cluster['공연시설코드']==theater]['ratio']-c1_aver)/c1_stdev
    final_resilience.append(resilience.iloc[0])
cluster['resilience']=final_resilience


In [ ]:
cluster

,공연시설코드,충격반응력,충격회복력,cluster,ratio,resilience
0,FC222007,0.999441,838.051600,1.0,7.019177,1.749359
1,FC222040,0.887045,2.596457,0.0,1.176063,-0.449641
2,FC222048,0.996858,53.153837,1.0,4.261825,0.737085
3,FC222049,0.000000,0.000000,0.0,-0.227898,-1.200015
4,FC222054,0.965359,2.864677,0.0,1.181252,-0.446868
...,...,...,...,...,...,...
133,FC223909,0.991397,16.383527,0.0,2.921606,0.483298
134,FC223941,0.000000,0.000000,1.0,-0.414639,-0.979729
135,FC224381,0.000000,0.000000,1.0,-0.414639,-0.979729
136,FC224454,0.000000,0.000000,1.0,-0.414639,-0.979729


In [ ]:
data = cluster[(cluster['충격회복력']!=0)&(np.isnan(cluster['cluster'])==False)]

In [ ]:
import numpy as np
all_scores=[]

scores=data['resilience']
max_score = max(scores)
min_score = min(scores)
scaled_scores = [(score - min_score) / (max_score - min_score) * 100 for score in scores]
percentiles = [np.percentile(scaled_scores, (score - min_score) / (max_score - min_score) * 100) for score in scores]
all_scores.append(percentiles)

for score, percentile in zip(scores, percentiles):
    print(f"Score: {score}, Percentile: {percentile}")


Score: 1.7493587957440262, Percentile: 64.25630356187325
Score: -0.4496413380033226, Percentile: 54.79025876370141
Score: 0.7370854618940551, Percentile: 59.6625740627516
Score: -0.4468677290898182, Percentile: 54.80056470767592
Score: 1.102484780137207, Percentile: 61.71849571811367
Score: -0.9881882393323802, Percentile: 52.35846257959355
Score: 0.5507246861372386, Percentile: 59.046450539504455
Score: -0.7713160548839608, Percentile: 53.95113151148649
Score: 0.7738612824022745, Percentile: 59.887069446058206
Score: 0.0987083252127934, Percentile: 56.894355076829434
Score: 0.17986478444958148, Percentile: 57.40498836174674
Score: 0.7053370055656389, Percentile: 59.43895693628988
Score: -0.23500340229082456, Percentile: 55.61357212245245
Score: 3.259391187651349, Percentile: 100.0
Score: 0.8738843066447279, Percentile: 60.4276335355049
Score: -0.02386649948368312, Percentile: 56.57423484970907
Score: -0.05260445143292152, Percentile: 56.51772227853635
Score: 1.6705262646511836, Percen

In [ ]:
data['회복점수']=all_scores[0]

C:\Users\ohke7\AppData\Local\Temp\ipykernel_18232\871335482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['회복점수']=all_scores[0]


In [ ]:
data

,공연시설코드,충격반응력,충격회복력,cluster,ratio,resilience,회복점수
0,FC222007,0.999441,838.051600,1.0,7.019177,1.749359,64.256304
1,FC222040,0.887045,2.596457,0.0,1.176063,-0.449641,54.790259
2,FC222048,0.996858,53.153837,1.0,4.261825,0.737085,59.662574
4,FC222054,0.965359,2.864677,0.0,1.181252,-0.446868,54.800565
5,FC222058,0.980903,51.365195,0.0,4.080114,1.102485,61.718496
...,...,...,...,...,...,...,...
129,FC223835,0.771788,3.490741,0.0,1.642747,-0.200213,55.621079
130,FC223889,0.981118,112.850000,0.0,4.867930,1.523548,63.052564
131,FC223890,0.757177,1.012102,0.0,0.359460,-0.886090,53.058773
132,FC223897,0.790950,5.823907,1.0,2.347460,0.034288,56.684318


In [ ]:
max(all_scores[0])

100.0

## 유형별 회복지수 확인

In [ ]:
data.groupby('cluster')['회복점수'].agg(['sum'])


,sum
cluster,
0.0,2949.017065
1.0,2541.016669


In [1]:
(2949.017065-2616.841216)/2616.841216*100


12.693771672847276

In [2]:
(2541.016669-2258.876440)/2258.876440*100

12.490290482643667

In [3]:
(2949.017065+2541.016669-2541.016669-2258.876440)/(2541.016669+2258.876440)*100

14.378249876147406